<h1 align='center'> বাংলায় মেশিন লার্নিংঃ মাল্টিপল ফিচার লিনিয়ার রিগ্রেশন </h1>
<p align='center'> K M Shariat Ullah <br> Student, Dept. of Electrical and Electronic Engineering, Shahjalal University of Science and Technology, Sylhet <br> E-mail: cast.shariat@gamil.com </p>

আগের নোটবুকে আমরা দেখেছিলাম যদি ফিচার কেবলমাত্র একটি হয় ($x$) তবে মডেল $\hat{y} = wx + b$। কিন্তু বাস্তবে তো কেবলমাত্র একটি ফিচার থাকা দুস্কর। কোনো একটি আউটপুটকে প্রভাবিত করতে পারে এমন ফ্যাক্টরের সংখ্যা একাধিক হবে এটাই স্বাভাবিক। যেমন নিচে আমরা একটি বাড়ির সাইজ, বেডরুমের সংখ্যা, ফ্লোরের সংখ্যা, বাড়ির বয়সের উপর ভিত্তি করে বাড়ির প্রাইজ দেখতে পাচ্ছি। আমাদেরকে এখন এই ফিচারগুলোর উপর ভিত্তি করে একটি মডেল তৈরি করতে হবে যাতে আমরা যে কোনো সাইজের বা যে কোনো বয়সের বাড়ির জন্য বা অন্য কোনো ফিচারের পার্টিকুলার মানের জন্য বাড়ির দাম কত হবে তা প্রেডিক্ট করতে পারে।

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
data = pd.read_csv('multiple_feature_house.csv')
data

,size,bedroom,floor,age,price
0,2104,5,1,45,460
1,1416,3,2,40,232
2,1534,3,2,30,315
3,852,2,1,36,178


এখানে আমাদের ফিচার চারটি <br>

$x_1$ : `Size of the Home` <br>
$x_2$ : `No. of Bedrooms` <br>
$x_3$ : `No. of Floors` <br>
$x_4$ : `Age of Home` <br>

আর এখানে টার্গেট একটি <br>

$y$ : `Price in USD 1000`

যদি আমরা একটি সরলরেখৈক মডেল বানাতে চাই তাহলে সকল ফিচারের পাওয়ার হতে হবে 1 অর্থাৎ লিনিয়ার রিগ্রেশন। আগেরটার সাথে এখানে পার্থক্য হচ্ছে শুধু এ জায়গায় যে সেখানে ফিচার ছিল একটা, এখানে একাধিক। তাহলে এখানে আমাদের মডেলটি হবে - 

$$f_{w_i,b}(x_i) = w_1 x_1 + w_2 x_2 + w_3 x_3 + w4 x_4 + b \tag{1}$$

weight দিয়ে মূলত কী বুঝায় তা নিয়ে আরেকটু ইনটুইটিভ আলোচনা করা উচিত বলে মনে হচ্ছে। weight মূলত কোন ফিচারের প্রভাব কেমন তা বুঝাতে ব্যবহার করা হয়। যেমন ধরেন আপনাদের থিওরি কোর্সের weight ল্যাব কোর্সের থেকে বেশি। weight মানে এখানে গুরুত্ববহ বেশি এটা বোঝানো হচ্ছে। তাই থিওরি কোর্সের Credit 3 আর ল্যাব কোর্সের Credit 1.5। তো যেই কোর্সের weight বেশি তার প্রভাব বেশি থাকবে বা সেটি আউটপুট তৈরিতে অন্যদের থেকে বেশি গুরুত্বপূর্ণ। যেমন উপরের চার ফিচারের জন্য আমরা পার্টিকুলারলি একটা মডেল কল্পনা করতে পারি - 

$$ f_{w_i,b}(x_i) = 0.1 x_1 + 4x_2 + 10 x_3 - 2x_4 + 80 $$

এই মডেলটি থেকে কিছু বিষয় উদ্ধার করে নেই আগে - 

<ul>
    <li>যদি কোনো বেডরুম না থাকে, কোনো ফ্লোর না থাকে, বাড়ির কোনো বয়স নেই, বাড়ির কোনো সাইজও নেই অর্থাৎ $x_1 = x_2 = x_3 = x_4 = 0$ তাহলে ঐ বাড়িটির দাম হবে ৮০ হাজার ডলার। এটাই b এর মান, তুলনা করে দেখেন</li>
    <li> বেডরুমের সংখ্যার সাথে 10 আছে ওয়েট হিসেবে। অন্যদিকে ফ্লোরের সংখ্যার সাথে 4 আছে ওয়েট হিসেবে। এ থেকে একটু তুলনা করেন। অর্থাৎ ফ্লোরের সংখ্যা যদি একটা বৃদ্ধি পায় তাহলে বাড়ির প্রাইজ চারগুণ হবে। আর যদি একটা বেডরুমের সংখ্যা বৃদ্ধি পায় তাহলে বাড়ির প্রাইজ দশগুণ বৃদ্ধি পাবে। এখানে তাহলে বেডরুমের সংখ্যাটা কততলা বাড়ি তার থেকেও বেশি গুরুত্বপূর্ণ। এজন্যই বেডরুমের সংখ্যার জন্য বড় ওয়েট নেওয়া হয়েছে। 
</ul>

#### Vectorization

যখন এধরণের একাধিক ফিচার নিয়ে মডেল তৈরি করা হবে তখন আমরা ভেক্টরাইজেশন নামে একটি পদ্ধতি অ্যাপ্লাই করব। এর জন্য আমরা দুইটা ভেক্টর $\vec{x}$ ও $\vec{w}$ সঙ্গায়িত করব। উপরের মডেলটির জন্য এ দুটি ভেক্টরকে এভাবে লেখা হবে - 

$$\vec{x} =  \begin{bmatrix} x_1 & x_2 & x_3 & x_4 \end{bmatrix} $$
$$\vec{w} =  \begin{bmatrix} 0.1 & 4 & 10 & -2 \end{bmatrix} $$

এমন ক্ষেত্রে আমাদের মডেলটিকে লেখা হবে এভাবে - 

$$f(\vec{w},\vec{x})=\vec{w} \cdot \vec{x} + b \tag{2}$$

ভেক্টরাইজেশন করার পর আগের পদ্ধতিতেই আমরা গ্র্যাডিয়েন্ট ডিসেন্ট করে কস্ট ফাংশনকে মিনিমাম করে এমন কিছু ওয়েট ও বায়াসের মান বের করব। 

In [3]:
x_train = np.array([
    [2104, 5, 1, 45],
    [1416, 3, 2, 40],
    [1534, 3, 2, 30],
    [852, 2, 1, 36]
])

y_train = np.array([460, 232, 315, 178])

In [4]:
b_init = 80.
w_init = np.array([0.1, 4, 10, -2])

y = np.dot(x_train, w_init) + b_init
y

array([230.4, 173.6, 205.4, 111.2])

In [17]:
# Actual target values
actual = np.array([460, 232, 315, 178])

# Number of examples (m) and features (n)
m, n = np.shape(x_train)

# Initialize weights and bias
w = np.zeros(n)
b = 0.0

# Set the learning rate and number of iterations
alpha = 0.01  # Reduced learning rate
num_iterations = 1000

# Gradient descent
for iteration in range(num_iterations):
    # Initialize gradients
    dj_dw = np.zeros(n)
    dj_db = 0.0
    
    # Compute gradients
    for i in range(m):
        # Calculate the error for the i-th example
        err = (np.dot(x_train[i], w) + b) - actual[i]
        
        # Accumulate gradient for each feature
        for j in range(n):
            dj_dw[j] += err * x_train[i, j]
        
        # Accumulate gradient for the bias
        dj_db += err

    # Average the accumulated gradients
    dj_dw /= m
    dj_db /= m

    # Update weights and bias
    w -= alpha * dj_dw
    b -= alpha * dj_db

# Display the final weights and bias
print("Predicted weights (w):", w)
print("Predicted bias (b):", b)

Predicted weights (w): [ 63.72111881  57.14324513 -25.75143498 -33.62061664]
Predicted bias (b): 296.23721051795474


আমরা এখানে weight এবং bias এর মান পেয়েছি। অর্থাৎ আমাদের হাইপোথিসিস মডেলটি হবে নিম্নরূপ - 
$$ \hat{y} = 63.72x_1 + 57.14x_2 - 25.75x_3 - 33.62x_4 + 296.24$$

এখন আমরা যদি $x_1 = 2000$ সাইজের একটি বাড়ির দাম কত বের করতে চাই যেটায় $x_2 = 3$ টা বেডরুম আছে, ফ্লোর আছে $x_3 = 2$ টা আর বাসাটির বয়স $x_4 = 30$ বছর তাহলে দাম হবে 

$$ \hat{y} = 63.72 \times 2000 + 57.14 \times 3 - 25.75 \times 2 - 33.62 \times 30 + 296.24$$ 
$$\Rightarrow \hat{y} = 126847.56$$

উপরের কোডটি রান করলে কারো কারো কোনো ডাটা শো নাও করতে পারে। এর কারণ হচ্ছে ডাটাগুলোর স্কেলিং খুব ছোট। মানে একটা দুইটা ডাটার মান যদি ১০, ১২ হয় আরেকটা ডাটার মান ১০১৮। মানে কিছু ডাটা খুব দূরে দূরে। এগুলোকে সমানভাবে স্কেলিং করা দরকার। একে বলা হয় `Renormalization`। অনেকভাবেই নরমালাইজেশন করা যায়। আমরা `Mean Normalization` করব। এটা করার পদ্ধতি হচ্ছে `x_train` এর নতুন মান বের করা নিচের ফর্মুলার মাধ্যমে। (যাদের পরিসংখ্যানে টুকিটাকি আইডিয়া আছে তারা বুঝেন এটি দিয়ে আসলে স্কেলিং কীভাবে বুঝানো হয়, বিস্তারিত আলোচনায় যাচ্ছি না)

$$x := \frac{x-\mu}{\sigma} \tag{3}$$

এখানে, $\mu $ : ডাটাসেটের গড় (Mean) ও $\sigma$ : আদর্শ বিচ্যুতি (Standard Deviation)। সে হিসেবে আগের কোডে `actual` ভেরিয়েবলের পরে শুধু নিচের লাইনটি অ্যাড করে দিলেই স্কেলিং হয়ে যাবে ও সুন্দর মান আসবে। 

`x_train = (x_train - np.mean(x_train, axis=0)) / np.std(x_train, axis=0)`
